### Custom dataset & Data load

In [1]:
from torch.utils.data import Dataset, DataLoader
from glob import glob
from PIL import Image

In [2]:
class PizzaDataset(Dataset):
    def __init__(self, path, subset, transforms=None):
        assert subset in ('train', 'val', 'test')
        self.pizza_files = glob(path+subset+'/pizza/*.jpg')
        self.not_pizza_files = glob(path+subset+'/not_pizza/*.jpg')
        self.transforms = transforms
        self.images = self.not_pizza_files + self.pizza_files
        self.labels = [0]*len(self.not_pizza_files) + [1]*len(self.pizza_files)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        images = Image.open(self.images[idx]).convert('RGB')
        labels = self.labels[idx]
        if self.transforms is not None:
            images = self.transforms(images)
        return images, labels

    def getfilename(self, idx):
        filename = Image.open(self.images[idx]).filename.split('/')[-1]
        return filename


def pizza_data_load(path, transforms, batch_size):
    
    train_loader = DataLoader(
        PizzaDataset(path=path, subset='train', transforms=transforms),
        batch_size=batch_size,
        shuffle=True,
        drop_last=False,
    )
    
    valid_loader = DataLoader(
        PizzaDataset(path=path, subset='val', transforms=transforms),
        batch_size=batch_size,
        shuffle=True,
        drop_last=False,
    )

    test_loader = DataLoader(
        PizzaDataset(path=path, subset='test', transforms=transforms),
        batch_size=batch_size,
        shuffle=True,
        drop_last=False,
    )
    return train_loader, valid_loader, test_loader